In [50]:
# Importing packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

In [9]:
# Read dataset

data = pd.read_excel(r'C:\Users\anant\OneDrive\Desktop\MBAN\MBAN 6110\Datasets\sfd_data_original.xlsx')

In [10]:
# Categorizing target variable 'Victims' into 'Low', 'Medium', 'High'

def categorize_victims(victims):
    if victims == 0:
        return 'Low'
    elif victims == 1:
        return 'Medium'
    else:
        return 'High'

In [11]:
data['Victims_Cat'] = data['VICTIMS'].apply(categorize_victims)

In [12]:
# Create a column to represent each shooting incident

data['Shootings'] = 1

In [13]:
# Convert 'OCC_DATE' to datetime

data['OCC_DATE'] = pd.to_datetime(data['OCC_DATE'])

In [14]:
# Sort by 'OCC_DATE' and 'NEIGHBOURHOOD_158'

data.sort_values(by=['NEIGHBOURHOOD_158', 'OCC_DATE'], inplace=True)

**Feature Engineering**

In [16]:
# Define the periods for the lag and rolling features

periods = [30, 60, 90, 180, 365]

# Create the lag and rolling features

for period in periods:
    data[f'Shootings_Last_{period}_Days'] = data.groupby('NEIGHBOURHOOD_158')['Shootings'].transform(lambda x: x.rolling(period).sum())
    data[f'Victims_Last_{period}_Days'] = data.groupby('NEIGHBOURHOOD_158')['VICTIMS'].transform(lambda x: x.rolling(period).sum())

**Outlier Removal**

In [17]:
# Calculate Q1, Q3, IQR

Q1_victims = data['VICTIMS'].quantile(0.25)
Q3_victims = data['VICTIMS'].quantile(0.75)
IQR_victims = Q3_victims - Q1_victims

# Define outliers and filter them out

outliers_victims = (data['VICTIMS'] < (Q1_victims - 3 * IQR_victims)) | (data['VICTIMS'] > (Q3_victims + 3 * IQR_victims))
data = data[~outliers_victims]

In [18]:
# Fill missing values in the lag and rolling features with 0

lag_and_rolling_cols = [col for col in data.columns if 'Last' in col]
data[lag_and_rolling_cols] = data[lag_and_rolling_cols].fillna(0)

In [19]:
# Map 'Victims_Cat' to numbers

mapping = {'Low': 0, 'Medium': 1, 'High': 2}
data['Victims_Cat'] = data['Victims_Cat'].map(mapping)

In [20]:
# One-hot encode categorical variables

data = pd.get_dummies(data, columns=['OCC_MONTH', 'OCC_DOW', 'OCC_TIME_RANGE', 'DIVISION', 'NEIGHBOURHOOD_158'])


In [21]:
# Define the input features and the target variable

X = data.drop(columns=['OCC_DATE', 'DEATH', 'INJURIES', 'HOOD_158', 'LONG_WGS84', 'LAT_WGS84', 'VICTIMS', 'Shootings', 'Victims_Cat'])
y = data['Victims_Cat']

In [22]:
# Get the date for the start of the last 30 days

test_start_date = data['OCC_DATE'].max() - pd.DateOffset(days=30)

In [23]:
# Split into training set and test set

X_train = X[data['OCC_DATE'] < test_start_date]
y_train = y[data['OCC_DATE'] < test_start_date]
X_test = X[data['OCC_DATE'] >= test_start_date]
y_test = y[data['OCC_DATE'] >= test_start_date]

First iteration - Random Forest

In [24]:
# Train a Random Forest model

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Make predictions on the test set

y_pred_rf = rf.predict(X_test)

# Calculate the performance metrics

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='macro')
recall_rf = recall_score(y_test, y_pred_rf, average='macro')
f1_rf = f1_score(y_test, y_pred_rf, average='macro')

accuracy_rf, precision_rf, recall_rf, f1_rf

C:\Users\anant\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.47368421052631576, 0.19999999999999998, 0.25, 0.22222222222222218)

Due to class imbalance, we're creating weights to ensure that this is accounted for when building the trees

In [25]:
# Calculate the scale_pos_weight parameter for each class

weights = y_train.value_counts().max() / y_train.value_counts()

Second iteration - Random Forest

In [26]:
# Train a Random Forest model

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train, sample_weight=weights[y_train])

# Make predictions on the test set

y_pred_rf = rf.predict(X_test)

# Calculate the performance metrics

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

accuracy_rf, precision_rf, recall_rf, f1_rf

C:\Users\anant\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.5789473684210527,
 0.3859649122807018,
 0.5789473684210527,
 0.46315789473684216)

In [27]:
# Standardize the features

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Using Principal Component Analysis to reduce dimensionality of the dataset

In [28]:
# Apply PCA to the training set

pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)

# Transform the test set

X_test_pca = pca.transform(X_test_scaled)

Third iteration - Random Forest

In [29]:
# Train a Random Forest model

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_pca, y_train, sample_weight=weights[y_train])

# Make predictions on the test set

y_pred_rf = rf.predict(X_test_pca)

# Calculate the performance metrics

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

accuracy_rf, precision_rf, recall_rf, f1_rf

C:\Users\anant\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.6842105263157895,
 0.7368421052631579,
 0.6842105263157895,
 0.5954887218045113)

Moving onto another model, XGBoost

In [30]:
# Train a XGBoost model

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train, y_train)

# Make predictions on the test set

y_pred_xgb = xgb.predict(X_test)

# Calculate the performance metrics

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb, average='macro')
recall_xgb = recall_score(y_test, y_pred_xgb, average='macro')
f1_xgb = f1_score(y_test, y_pred_xgb, average='macro')

accuracy_xgb, precision_xgb, recall_xgb, f1_xgb

c:\Users\anant\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\anant\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.3157894736842105,
 0.20370370370370372,
 0.19444444444444445,
 0.19595959595959597)

Second iteration - XGBoost

In [31]:
# Train a XGBoost model with scale_pos_weight

xgb_balanced = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_balanced.fit(X_train, y_train, sample_weight=weights[y_train])

# Make predictions on the test set

y_pred_xgb_balanced = xgb_balanced.predict(X_test)

# Calculate the performance metrics

accuracy_xgb_balanced = accuracy_score(y_test, y_pred_xgb_balanced)
precision_xgb_balanced = precision_score(y_test, y_pred_xgb_balanced, average='macro')
recall_xgb_balanced = recall_score(y_test, y_pred_xgb_balanced, average='macro')
f1_xgb_balanced = f1_score(y_test, y_pred_xgb_balanced, average='macro')

accuracy_xgb_balanced, precision_xgb_balanced, recall_xgb_balanced, f1_xgb_balanced

c:\Users\anant\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


(0.5263157894736842,
 0.5333333333333333,
 0.5833333333333334,
 0.5555555555555555)

Using GridSearchCV for Hyperparameter tuning

In [ ]:
# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 6, 10],
    'min_child_weight': [1, 5, 10],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'n_estimators' : [100, 200, 500],
}

# Create a XGBoost classifier
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Create the grid search object
scorer = make_scorer(f1_score, average='weighted')
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring=scorer, verbose=True)

# Fit the grid search object to the data
grid_search.fit(X_train_pca, y_train)

# Print the best parameters and the best score
print(grid_search.best_params_)
print(grid_search.best_score_)

Applying best parameters and training our XGBoost model

In [33]:
# Apply PCA to the standardized training set

pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)

# Transform the test set using the PCA model
X_test_pca = pca.transform(X_test_scaled)

# Train an XGBoost model on the PCA-transformed training set
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_pca, y_train, sample_weight=weights[y_train])

# Make predictions on the PCA-transformed test set
y_pred = xgb.predict(X_test_pca)

# Calculate the performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

c:\Users\anant\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Accuracy: 0.7368421052631579
Precision: 0.7616099071207431
Recall: 0.7368421052631579
F1 Score: 0.6805807622504537


C:\Users\anant\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Just as an experiment, we wanted to see how our model would perform with only the top 20 neighbourhoods

In [42]:
# Reading the file again

df = pd.read_excel(r'C:\Users\anant\OneDrive\Desktop\MBAN\MBAN 6110\Datasets\sfd_data_original.xlsx')

In [43]:
# Converting to datetime datatype

df['OCC_DATE'] = pd.to_datetime(df['OCC_DATE'])

In [44]:
# Filtering for only top 20 neighbourhoods where shootings have occured

top_20_neighbourhoods = df['NEIGHBOURHOOD_158'].value_counts().head(20).index
df['NEIGHBOURHOOD_158'] = df['NEIGHBOURHOOD_158'].where(df['NEIGHBOURHOOD_158'].isin(top_20_neighbourhoods), 'Other')

In [45]:
# Thirty day period feature

df['thirty_day_period'] = df['OCC_DATE'].dt.to_period('30D')

C:\Users\anant\AppData\Local\Temp\ipykernel_18000\3287101991.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['thirty_day_period'] = df['OCC_DATE'].dt.to_period('30D')


In [46]:
# Go back to the original data_grouped dataframe

data_grouped = df.groupby(['thirty_day_period', 'NEIGHBOURHOOD_158', 'OCC_TIME_RANGE']).size().reset_index(name='incidents')
data_grouped['incidents'] = data_grouped['incidents'].shift(-1)
data_grouped = data_grouped[data_grouped['thirty_day_period'] < data_grouped['thirty_day_period'].max()]

In [47]:
# Group the less frequent neighbourhoods into an 'Other' category in the aggregated data

data_grouped['NEIGHBOURHOOD_158'] = data_grouped['NEIGHBOURHOOD_158'].where(data_grouped['NEIGHBOURHOOD_158'].isin(top_20_neighbourhoods), 'Other')

In [48]:
# One-hot encode the 'NEIGHBOURHOOD_158' and 'OCC_TIME_RANGE' columns in the aggregated data

data_grouped_encoded = pd.get_dummies(data_grouped, columns=['NEIGHBOURHOOD_158', 'OCC_TIME_RANGE'])

In [49]:
# Determine the row to split on

split_row = int(len(data_grouped_encoded) * 0.8)

# Sort the data by 'three_day_period'

data_grouped_encoded = data_grouped_encoded.sort_values('thirty_day_period')

# Split the data into a training set and a test set

train = data_grouped_encoded.iloc[:split_row].copy()
test = data_grouped_encoded.iloc[split_row:].copy()

In [ ]:
# Define a function to transform the 'incidents' column

def transform_incidents(incidents):
    if incidents <= low_threshold:
        return 'Low'
    elif incidents <= medium_threshold:
        return 'Medium'
    else:
        return 'High'

# Compute the 33rd and 66th percentiles of the 'incidents' column in the training set

low_threshold = train['incidents'].quantile(0.33)
medium_threshold = train['incidents'].quantile(0.66)

In [ ]:
# Transform the 'incidents' column in the training set and the test set

train['incidents'] = train['incidents'].apply(transform_incidents)
test['incidents'] = test['incidents'].apply(transform_incidents)
X_train = train.drop(columns=['incidents', 'thirty_day_period'])
y_train = train['incidents']
X_test = test.drop(columns=['incidents', 'thirty_day_period'])
y_test = test['incidents']

In [51]:
# Create a gradient boosting classifier

clf = GradientBoostingClassifier(random_state=42)

# Fit the classifier on the training data

clf.fit(X_train, y_train)

# Use the classifier to predict the classes of the test data

y_pred = clf.predict(X_test)


In [53]:
accuracy_xgb = accuracy_score(y_test, y_pred)
precision_xgb = precision_score(y_test, y_pred, average='macro')
recall_xgb = recall_score(y_test, y_pred, average='macro')
f1_xgb = f1_score(y_test, y_pred, average='macro')